In [9]:
%cd ..

/home/lingze/embedding_fusion


In [10]:
from model.base import CompositeModel, FeatureEncodingPart, NodeRepresentationPart
from relbench.modeling.nn import HeteroTemporalEncoder

In [1]:
import relbench
from relbench.base import Table, Database, Dataset, EntityTask
from relbench.datasets import get_dataset
from relbench.tasks import get_task
from relbench.base import BaseTask
from torch_geometric.seed import seed_everything
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph
from relbench.modeling.graph import get_node_train_table_input
from torch_geometric.loader import NeighborLoader

import os
import math
import numpy as np
from tqdm import tqdm
import copy

import torch
from torch import Tensor
import torch_geometric
import torch_frame

from torch_frame.config.text_embedder import TextEmbedderConfig
from typing import List, Optional
from sentence_transformers import SentenceTransformer

from torch.nn import L1Loss, BCEWithLogitsLoss
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
dataset = get_dataset(name="rel-trial", download=True)
db = dataset.get_db()
task_a = get_task("rel-trial", "study-outcome", download = True)
task_b = get_task("rel-trial", "site-success", download = True)

Loading Database object from /home/lingze/.cache/relbench/rel-trial/db...
Done in 9.62 seconds.


In [3]:
col_to_stype_dict = get_stype_proposal(db)

class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)


root_dir = "/home/lingze/embedding_fusion/data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-trial_materialized_cache"
    ),  # store materialized graph for convenience
)

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
def generate_loader_dict(task: BaseTask) -> dict:
    loader_dict = {}
    for split, table in [
        ("train", task.get_table("train")),
        ("val",task.get_table("val")),
        ("test", task.get_table("test")),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )
        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[
                128 for i in range(2)
            ],  # we sample subgraphs of depth 2, 128 neighbors per node.
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=512,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )
    return loader_dict

In [5]:
taska_loader_dict = generate_loader_dict(task_a)
taskb_loader_dict = generate_loader_dict(task_b)

In [6]:
@torch.no_grad()
def valid(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    pred_hat_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
        pred_hat_list.append(batch[task.entity_table].y.detach().cpu())
    return torch.cat(pred_list, dim=0), torch.cat(pred_hat_list, dim=0)

@torch.no_grad()
def test(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0)

In [31]:
channels = 128

task_a_temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
)

task_a_feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)


task_a_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.2
)



task_a_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    dropout=0.2,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=task_a_feat_encoder,
    node_encoder=task_a_node_encoder,
    temporal_encoder=task_a_temporal_encoder
)

In [32]:
# Train Task A
optimizer = torch.optim.Adam(task_a_model.parameters(), lr=0.005)
epochs = 15
loss_fn = BCEWithLogitsLoss()
tune_metric = "auroc"
higher_is_better = True

In [33]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
task_a_model.to(device)
best_epoch = 0
task_a_model.reset_parameters()
for epoch in range(1, epochs + 1):
    task_a_model.train()
    loss_accum = count_accum = 0
    for batch in tqdm(taska_loader_dict["train"]):
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = task_a_model(
            batch,
            task_a.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        loss = loss_fn(pred, batch[task_a.entity_table].y.float())
        
        loss.backward()
        optimizer.step()
        
        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)
    
    train_loss = loss_accum / count_accum
    val_logits = test(taska_loader_dict["val"], task_a_model, task_a)
    val_logits = torch.sigmoid(val_logits).numpy()
    
    val_pred = (val_logits > 0.5).astype(int)
    val_pred_hat = task_a.get_table("val").df[task_a.target_col].to_numpy()
    val_metrics = {
            "auroc": roc_auc_score(val_pred_hat, val_logits),
        "accuracy": accuracy_score(val_pred_hat, val_pred),
        "precision": precision_score(val_pred_hat, val_pred),
        "recall": recall_score(val_pred_hat, val_pred),
        "f1": f1_score(val_pred_hat, val_pred),
    }
    
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_epoch = epoch
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(task_a_model.state_dict())

best_epoch

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:05<00:00,  4.11it/s]


Epoch: 01, Train loss: 0.6894868534068893, Val metrics: {'auroc': 0.4877367214828515, 'accuracy': 0.584375, 'precision': 0.584375, 'recall': 1.0, 'f1': 0.73767258382643}


100%|██████████| 24/24 [00:04<00:00,  4.81it/s]


Epoch: 02, Train loss: 0.6564565871046925, Val metrics: {'auroc': 0.5614973262032086, 'accuracy': 0.584375, 'precision': 0.584375, 'recall': 1.0, 'f1': 0.73767258382643}


...

100%|██████████| 24/24 [00:05<00:00,  4.66it/s]


Epoch: 14, Train loss: 0.48224579991251904, Val metrics: {'auroc': 0.6213126398884913, 'accuracy': 0.621875, 'precision': 0.651840490797546, 'recall': 0.7575757575757576, 'f1': 0.7007419620774938}


100%|██████████| 24/24 [00:05<00:00,  4.76it/s]


Epoch: 15, Train loss: 0.46138886097313586, Val metrics: {'auroc': 0.6300197910105031, 'accuracy': 0.5770833333333333, 'precision': 0.6565656565656566, 'recall': 0.5793226381461676, 'f1': 0.615530303030303}


5

In [34]:
# test task A
task_a_model.load_state_dict(state_dict)
test_logits = test(taska_loader_dict["test"], task_a_model, task_a)
test_logits =  torch.sigmoid(test_logits).numpy()

test_pred = (test_logits > 0.5).astype(int)
test_pred_hat = task_a.get_table("test", mask_input_cols = False).df[task_a.target_col].to_numpy()
test_metrics = {
    "auroc": roc_auc_score(test_pred_hat, test_logits),
    "accuracy": accuracy_score(test_pred_hat, test_pred),
    "precision": precision_score(test_pred_hat, test_pred),
    "recall": recall_score(test_pred_hat, test_pred),
    "f1score": f1_score(test_pred_hat, test_pred),
}
test_metrics

{'auroc': 0.6789013596793917,
 'accuracy': 0.5878787878787879,
 'precision': 0.5868772782503038,
 'recall': 1.0,
 'f1score': 0.7396630934150077}

In [35]:
# save the individual model state
torch.save(task_a_model.state_dict(), "task_a_model.pth")

In [53]:
# train Task B
task_b_temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
)

task_b_feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)


task_b_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.4
)

task_b_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    dropout=0.3,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=task_b_feat_encoder,
    node_encoder=task_b_node_encoder,
    temporal_encoder=task_b_temporal_encoder
)

In [54]:
optimizer = torch.optim.Adam(task_b_model.parameters(), lr=0.005)
epochs = 20
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False

In [58]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
task_b_model.to(device)
best_epoch = 0
early_stop = 40
# train
task_b_model.reset_parameters()
for epoch in range(1, epochs + 1):
    task_b_model.train()
    
    cnt = 0
    loss_accum = count_accum = 0
    for batch in tqdm(taskb_loader_dict["train"]):
        cnt += 1
        if cnt > early_stop:
            break
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = task_b_model(
            batch,
            task_b.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        loss = loss_fn(pred, batch[task_b.entity_table].y.float())
        
        loss.backward()
        optimizer.step()
        
        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    train_loss = loss_accum / count_accum
    val_pred_hat = task_b.get_table("val").df[task_b.target_col].to_numpy()
    val_logits = test(taskb_loader_dict["val"], task_b_model, task_b)
    val_logits = val_logits.numpy()
    
    val_metrics = {
        "mae": mean_absolute_error(val_pred_hat, val_logits),
        "r2": r2_score(val_pred_hat, val_logits),
        "rmse": root_mean_squared_error(val_pred_hat, val_logits),
    }
    
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    
    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_epoch = epoch
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(task_b_model.state_dict())

best_epoch

  0%|          | 0/296 [00:00<?, ?it/s]

 14%|█▎        | 40/296 [00:06<00:41,  6.17it/s]


Epoch: 01, Train loss: 0.4110280603170395, Val metrics: {'mae': 0.3936864982174104, 'r2': -0.30282323912695497, 'rmse': 0.5452460380940995}


 14%|█▎        | 40/296 [00:06<00:42,  5.96it/s]


Epoch: 02, Train loss: 0.24289779141545295, Val metrics: {'mae': 0.384304029025163, 'r2': -0.2736703055791698, 'rmse': 0.5391111102901006}


...

 14%|█▎        | 40/296 [00:06<00:39,  6.45it/s]


Epoch: 19, Train loss: 0.17650280594825746, Val metrics: {'mae': 0.4072830869679515, 'r2': -0.48362593122006525, 'rmse': 0.5818512584067168}


 14%|█▎        | 40/296 [00:06<00:42,  5.99it/s]


Epoch: 20, Train loss: 0.18459568172693253, Val metrics: {'mae': 0.3996612915887673, 'r2': -0.5034183352771859, 'rmse': 0.585719511441545}


6

In [61]:
# test
task_b_model.load_state_dict(state_dict)
logits = test(taskb_loader_dict["test"], task_b_model, task_b)
logits = logits.numpy()
pred_hat = task_b.get_table("test", mask_input_cols=False).df[task_b.target_col].to_numpy()
test_metrics = {
        "mae": mean_absolute_error(pred_hat, logits),
        "r2": r2_score(pred_hat, logits),
        "rmse": root_mean_squared_error(pred_hat, logits),
}
test_metrics

{'mae': 0.40999596248485687,
 'r2': -0.5214476645866393,
 'rmse': 0.5935707603008625}

In [62]:
# save the individual model state
torch.save(task_b_model.state_dict(), "task_b_model.pth")

: 